In [ ]:
import jax
from jax import config
config.update("jax_enable_x64", True)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline


import tempfile
import os, shutil
import logging
import zipfile

from viperleed_jax.data_structures import ReferenceData
from viperleed_jax.tensor_calculator import TensorLEEDCalculator
from viperleed_jax.files.phaseshifts import Phaseshifts

jax.devices()

In [ ]:
from viperleed_jax.from_state import calculator_from_state
from viperleed_jax.parameter_space import ParameterSpace

In [ ]:
from viperleed.calc.files.displacements import readDISPLACEMENTS
from viperleed.calc import LOGGER as logger
from viperleed.calc.files.phaseshifts import readPHASESHIFTS
from viperleed.calc.files.iorfactor import beamlist_to_array

In [ ]:
origin_path = Path('../tests/test_data/unrelaxed_frozen_for_testing/')
tensor_path = Path('../large_files/unrelaxed_frozen_for_testing/Tensors_001.zip')

calculator, slab, rpars, ref_data, phaseshifts = calculator_from_state(origin_path, tensor_path, l_max=10)

In [ ]:
parameter_space = ParameterSpace(slab)

## GEOMETRY
# Fix layers 0 and 1
parameter_space.geo_params.fix_layer(0, z_offset=0.)
parameter_space.geo_params.fix_layer(1, z_offset=0.)

# symmetry constrained xyz movements ± 0.15 A for layer 2
for param in [p for p in parameter_space.geo_params.terminal_params if p.bound is None]:
    param.set_bound(geo_parameters.GeoParamBound(-0.15, + 0.15))

## VIBRATIONS
# fix *_def sites (O_def, Fe_def)
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_def')]:
    parameter_space.vib_params.fix_site_element(param.site_element, None) # None fixes to the default value

# the rest can vary ± 0.05 A
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_surf')]:
    param.set_bound(vib_parameters.VibParamBound(-0.1, + 0.1))


## CHEMISTRY
# no free parameters
parameter_space.occ_params.remove_remaining_vacancies()

# V0R
# set ± 2 eV
parameter_space.v0r_param.set_bound(v0r_parameters.V0rParamBound(-2., +2.))

In [ ]:
# benchmark

In [ ]:
# ref R-factor